# FT-ICR analysis - PART 1

Created on 01 March 2020 for the Pentatrap experiment

@author: Menno Door<br>
@contact: door+fticr@mpi-k.de<br>
@license: MIT license

### Refenences :

used toolkit of Jonas Karthein for PI-ICR as template :) thank you!!<br>

For references reguarding the theory behind ft-icr detection and analysis methods, please check the references.md file.

### Introduction

This part of the analysis does the conversion of the raw data to penning trap eigenfrequency data. You will be able to group, filter and average data and then fit spectra and/or evaluate pnp phase data to determine these frequencies.

### Important remark:

This is more a visualization thing, especially for bigger data sets i would recommend to do this using a normal python script including some multiprocessing.

Please be patient with this one! Gathering information on paths in fhds structures may take a while, loading data too. So please look at the circle on the top right corner (jupyter notebook), if its filled, there is still something going on and you should just wait.

### Requirements:

The following code was written for Python 3.7/3.8. The required libraries are listed below with a rough description for their task in the code.

    fhds (inhouse data storage, https://git.mpi-hd.mpg.de/Pentatrap/fhds)
    h5py (data storage, easier to distribute)
    pandas (data organisation, calculation and visualization)
    numpy (calculation)
    matplotlib (plotting)
    scipy (chi square fitting)
    jupyter (Python notebook environment)
    ipywidgets (https://ipywidgets.readthedocs.io/en/latest/user_install.html)
    plotly (plotting, https://github.com/plotly/plotly.py#installation)
    qgrid (data visualization, https://github.com/quantopian/qgrid#installation)

### ToDo 

- check for phase correlation here with single phases, short and long phases and unwrap.

In [ ]:
# standard libs
from datetime import datetime, timedelta
from pathlib import Path
import os, json
from pprint import pprint

# math and data
import numpy as np
import scipy
import pandas as pd
import h5py

# visualization
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
#display(HTML("<style>div.output_scroll { height: 150em; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
%matplotlib inline
import matplotlib.pyplot as plt
import qgrid
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
py.init_notebook_mode()
plt.rcParams['figure.figsize'] = (10, 4)

%load_ext autoreload
%autoreload 2

# this package
from fhds import fhds, interface
from fticr_toolkit import *
#from trap_control import fitaxialClass

### Step 1: create simulated data

config first

In [ ]:
# frequency in Hz, phase in radian
freq = 12315.14345
acc_times = [0.1, 0.5, 1.1, 2.1, 5.1, 10.1, 20.1, 40.1]
phase_offset = 4.2
number_avg = 5
phase_jitter = 15/180*np.pi
acc_time_randomshift = 1.5e-6 
ion = '187Re29+'
mc = 1
cycle = 1
trap = 2
pos = 'position_1'
time = datetime.now()

In [ ]:
pre_unwrap_phase = pd.DataFrame(columns=['mcycle', 'cycle', 'position', 'trap', 'acc_time', 'time', 'phase'])

for acc in acc_times:
    acc_shift = acc_time_randomshift*(float(np.random.rand(1))-0.5)
    print(acc, acc_shift)
    for i in range(number_avg):
        N = freq*(acc+acc_shift)
        pjitter = float(np.random.rand(1)*phase_jitter)
        phase = ((N - int(N))*2*np.pi + pjitter + phase_offset) % (2*np.pi)
        itime = time + timedelta(minutes=i)
        new_row = pd.Series([mc, cycle, pos, trap, acc, itime, phase], index=pre_unwrap_phase.columns )
        pre_unwrap_phase = pre_unwrap_phase.append(new_row, ignore_index=True)
        
#display(pre_unwrap_phase)

In [ ]:
print(" >>> UNWRAP DATA >>> ")
pre_unwrap_phase["acc_time"] = pre_unwrap_phase["acc_time"].astype(str)
fig = px.line(pre_unwrap_phase, x="time", y="phase", facet_col="trap", facet_row="position", color="acc_time", hover_data=['mcycle', 'cycle', 'position'])
#fig.update_yaxes(matches=None)
fig.show()
pre_unwrap_phase["acc_time"] = pre_unwrap_phase["acc_time"].astype(float)

### Step 4: N determination

The unwrap data (in this case just for the nu_p phase measurement) is used to calculate the total N of osciallations during the phase accumulation time in the later measurement

The method n_determination.fit_N will unwrap the phases for each acc_time, average, substract the reference phases from all the other measured phases and then determine the N by calculating Ns for a 1 Hz range around the guessed frequency and searches for the minimum. A plot of the Ns and the found minimum will be plotted to check the results.

The result will be a dataFrame including N, end_phase and frequency for all main cycles, traps and positiions.

In [ ]:
# each ion in each trap its own N determination, for every main cycle! (no grouping here, wouldn't make any sense)

columns = ["mcycle", "trap", "position", "N", "end_phase", "nu_p", "ion", "time", "max_acc_time"]
nu_p_N = pd.DataFrame(columns = columns)

print(" >>> mc", mc, "trap", trap, "pos", pos, " <<< ")
N, end_phase, nu_p, mean_time, max_acc_time = phase_analysis.determine_N(pre_unwrap_phase, freq, resolution=None, nu_range=2, show=True)
new_row = pd.Series([mc, trap, pos, N, end_phase, nu_p, ion, mean_time, max_acc_time], index=nu_p_N.columns )
nu_p_N = nu_p_N.append(new_row, ignore_index=True)

# show results and save to csv in results folder
display(nu_p_N)

In [ ]:
mag = 10e3
red = 21e6
axi = 500e3

mag2 = mag - 2
red2 = red + 2e5
axi2 = axi

print(red2/red)
print(176/174)

detune = np.arange(-10, 10, 10)
R = []
for det in detune:
    cyc1 = np.sqrt((mag+det)**2 + axi**2 + red**2)
    cyc2 = np.sqrt((mag2+det*1.1)**2 + axi2**2 + red2**2)
    R.append(cyc1/cyc2)

R = np.asarray(R)
R -= R.min()
plt.plot(detune, R)
plt.show()